In [1]:
# Import Libraries:

import time
start_time = time.time()
import datetime
import os
import re
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
# filter warning codes
warnings.filterwarnings("ignore")


In [2]:
# Setting up directory:
path = os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Raw Data")
path_ = os.getcwd()
path_

'C:\\ABI\\Q2_Competitive\\Ad_Spend\\Raw Data'

In [3]:
# Obtaining the files:
files = os.listdir(path_)
print("The number of markets",len(files))
files

The number of markets 29


['Argentina_Q2_2023 Competitive Request_Group I.xlsx',
 'Belgium_ABI Global Competitive _Group I.xlsx',
 'Bolivia_Q2_2023 Competitive Request_Group II.xlsx',
 'Brazil_2023 Competitive Q1 Q2_vs2_ Group I_new.xlsx',
 'Canada_Q2_2023 Competitive Request _Group I - New.xlsx',
 'Chile_Q2_2023 Competitive Request_Group II.xlsx',
 'China_Q2_2023 Competitive Request_Group I.xlsx',
 'Colombia_Q2_2023 ABI Global Competitive_Group I - New.xlsx',
 'DR_Q2_2023 Competitive Request_Group II.xlsx',
 'Ecuador_Q2_2023 Competitive Request_Group II.xlsx',
 'El Salvador_Q2_2023 Competitive Request_Group II.xlsx',
 'France_ABI Global Competitive _Group I.xlsx',
 'Germany_ABI Global Competitive Request Group I.xlsx',
 'Guatemala_Q2_2023 Competitive Request_Group II.xlsx',
 'Hondurus_Q2_2023 Competitive Request_group II.xlsx',
 'India_Q2_2023 Competitive Request_Group I.xlsx',
 'Italy_ABI Global Competitive_Group I.xlsx',
 'Japan_Q2_2023 Competitive Request_Group II.xlsx',
 'Mexico_Q2 _2023 Competitive Reques

In [4]:
## Initialize Empty dataframe:
df_initial = pd.DataFrame()
for f in files:
    data = pd.read_excel(f)
    df_initial = df_initial.append(data)

In [5]:
df_initial.head()

,Country,Data Source,Advertiser (parent company),Brand,Category,Media type,Year,Month,Spend (Local Currency),Exchange Rate,Spend (US Currency)
0,Argentina,IBOPE - AdMedia,PEPSI COLA,GATORADE.,ENERGY DRINK,TV Cable,2023,6,3181.423507,253.93,12.528742
1,Argentina,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,2268.824968,253.93,8.934844
2,Argentina,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,4537.649935,253.93,17.869688
3,Argentina,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,5834.121345,253.93,22.975313
4,Argentina,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,2106.766041,253.93,8.296641


In [6]:
df_initial.shape

(494986, 11)

In [7]:
df = df_initial.copy()

In [8]:
df.columns

Index(['Country', 'Data Source', 'Advertiser (parent company)', 'Brand',
       'Category', 'Media type', 'Year', 'Month', 'Spend (Local Currency)',
       'Exchange Rate', 'Spend (US Currency)'],
      dtype='object')

In [9]:
#os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Processed Data")
#print(os.getcwd())
#df.to_excel("test output.xlsx", index=False)

In [10]:
# Renameing features: for easy indexing in python
new_columns = ['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency']
df.columns = new_columns
df.columns

Index(['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency'],
      dtype='object')

In [11]:
## Making the String in upper case:
for col in df.columns[1:-7]:
    #print(col)
    df[col] = df[col].str.upper()

### Available Markets

In [12]:
df['Country'].unique()

array(['Argentina', 'Belgium', 'Bolivia', 'Brazil', 'Canada', 'Chile',
       'China', 'Colombia', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'France', 'Germany', 'Guatemala', 'Honduras',
       'India', 'Italy', 'Japan', 'Mexico', 'Netherlands', 'Panama',
       'Paraguay', 'Peru', 'South Africa', 'South Korea', 'UK', 'Uruguay',
       'US', 'Vietnam'], dtype=object)

In [13]:
## Creating requried columns with null values
df["Category_Standardized"] = np.nan
df["Advertiser_parent_standarized"] = np.nan
df["Brand_standarization"] = np.nan
df['Parent_competitor'] = np.nan
df["Top_Brand_Competitors_and_ABI"] = np.nan
df["Focused_Brand_Competitors"] = np.nan
df['Top10_Brands_by_Market'] = np.nan
df["Media_Summary"] = np.nan
df["Media_type_standarized"] = np.nan
df["Month_number"] = np.nan
df['Month_standarization'] = np.nan
df["Focused_Global_Brand"] = np.nan
df["Global_Parent"] = np.nan
df["ABI_Parent_SOS"] = np.nan
df["Heineken_Brand_SOS"] = np.nan
df["Amstel_Brand_SOS"] = np.nan
df["Digital_Traditional"] = np.nan
df["Heineken_Parent_SOS"] = np.nan
df["Brand_Rank_by_Country"] = np.nan
df["Market_Group"] = np.nan

### Category (standardized)

In [14]:
#df['Category'].unique()

In [15]:
# Drink Categorization:
beer = ["MALTAS", 'MALTA', "BEER", "BEER - OTHER", "CERVEZA", "CERVEZAS", "CERVEZAS ABI", 
        "CERVEZAS ARTESANALES", "CERVEZAS CCC", "CERVEZAS RESTANTES", "EVENTS DRINKS", "INST. BAVARIA", 'BEER RANGES',
       'Light & Low Alcohol Beer & Ale', 'Regular Beer & Ale', 'Cerveza', "Alcoholic Drink - Brand Building", 
       "Alcoholic Drink - Sports Sponsorship", "Alcoholic Drink - Sports Sponsorship", "Ales & Bitters", 
       "Beer Lager Cider - Multi Product", "Lagers", "Stouts", 'BIERES MULTI-PRODUITS', 'BIERES SPECIALES', 'PILS',
       'PILS DE LUXE', 'Lager', 'Alcoholic Drink-Brand Building', 'Ales&Bitters', 'Stout', 'Ready Mixed/alcopops ', 
       'BIRRE ALCOOLICHE', 'BIRRE ANALCOOLICHE', 'EVENTI E SPETTACOLI']


beyond_beer = ["SELTZER", 'beyond Beer', "COOLERS, READY TO DRINK", "Ready Mixed/alcopops"]

non_alcoholic_beer = ['Non Alc', "Alcohol Free Beer Lager Cider", 'Beer Non Alcoholic,- Arm']

other_alcohol = ["Cider/perry", "ENERGY DRINK", "COLAS", "E-COMMERCE", "FERNETS", "GIN", "INSTITUC BEBIDAS ALCOHOLICAS", 
                 "JUGOS NATURALES", "LIMON DIET", "LINEA SABORES", "MANZANA", "NARANJA", "SIDRAS"," VINOS ALTA GAMA", 
                 "VINOS BAJA GAMA", "VINOS ESPUMANTES", "VINOS MEDIA GAMA", "Liqueur", "LIQUEURS"," Non-Beer", 
                 "OTHER", "Other Alcoholic Drinks", "OTHER LIQUORS", "OTROS LICORES", "RON", "RTD ALCOHOLIC BEVERAGES",
                 "RTD MIXES", "RUM", "Soda", "SODAS", "SPIRITS", "SPIRITS - OTHER", "UVA", "VINO", "VINOS", 
                 "VINOS FRIZZANTE", "VODKA", "WHISKY", "WHISKYS", "WINE", "WINE - OTHER", "WINE/SPARKLING WINE",
                 "AGUAS MINERALES Y MINERALIZADA", "AGUAS SABORIZADAS CON GAS", "AGUAS SABORIZADAS LINEA",
                 "AGUAS SABORIZADAS SIN GAS", "ALCOHOLIC BEVERAGES", "CAMPAÑAS DE CONSUMO ALCOHOLIC", "CHAMPAGNE",
                 "Destilled Spirit", "INSTITUC BEBIDAS SIN ALCOHOL", "LICORES", "LIMA LIMON, CITRUS",
                 "LINEA BEBIDAS ALCOHOLICAS", "LINEA BEBIDAS SIN ALCOHOL", "APERITIV SIN ALCOHOL Y VEGETAL",
                 "BEBIDAS ALCOHOLICAS PREPARADAS", "BEBIDAS ALCOHOLICAS SABORIZADAS", "DISTRIBUID BEBIDAS ALCOHOLICAS",
                 "FLAVORED ALCOHOLIC BEVERAGES", "GRAPAS", "IND. LICORERA", "AGUARDIENTE", "ALCOPOPS", "APERITIVOS",
                 "BEBIDAS DE SOJA", "CIDER", "CIDER - OTHER", "COCTELES", "COLAS DIET", "EMBOTELLADORAS", "FRUTAS",
                 "JUGOS EN POLVO PARA PREPARAR", 'VINOS ALTA GAMA', 'NON-BEER', 'LIQUEUR', 'DESTILLED SPIRIT', 
                 'OTHER ALCOHOLIC DRINKS', 'SODA', 'Cider', 'Wine', 'Alcoholic Beverages: Comb Copy & NEC', 'Non-Beer', 
                 'AGUA ARDIENTE', 'COGNACS', 'RONES', 'TIENDAS DE LICORES', 'VODKAS', 'WHISKEY', 
                 'COCKTAILES/MEZCLADORES', 'AUTRES MANIFESTATIONS', 'MANIFESTATIONS SPORTIVES', 
                 'SPIRITUEUX MULTI-PRODUITS', ]

df["Category_Standardized"] = df['Category'].copy()
df["Category_Standardized"] = df["Category_Standardized"].replace(beer, "Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(beyond_beer, "Beyond Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(other_alcohol, "Other Alcohol")
df["Category_Standardized"] = df["Category_Standardized"].replace(non_alcoholic_beer, "Non Alcoholic Beer")
df['Category_Standardized'].unique()

array(['Other Alcohol', 'Beer', 'Beyond Beer', 'Non Alcoholic Beer'],
      dtype=object)

### Advertiser (standardized)

In [16]:
df["Advertiser_parent_standarized"] = df["Advertiser_parent_company"].copy()

## Argentina:
#(Changed from Heineken Parent to heineken)
df.loc[df['Advertiser_parent_standarized'] == "CCU SA", 'Advertiser_parent_standarized'] = "CCU SA (Heineken Parent)"  ## Changed
df.loc[df['Advertiser_parent_standarized'] == "QUILMES", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GOOSE ISLAND BEER COMPANY", 'Advertiser_parent_standarized'] = "ABI"
#Bolivia
df.loc[df['Advertiser_parent_standarized'] == "ABINVEB", 'Advertiser_parent_standarized'] = "ABI"
##Chile
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA CHILE", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CCU", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)" ## changed
df.loc[df['Advertiser_parent_standarized'] == "PATAGONIA (CERVEZA)", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MOLSON COORS CHILE", 'Advertiser_parent_standarized'] = "MOLSON COORS"
##China
df.loc[df['Advertiser_parent_standarized'] == "Carlsberg Group".upper(), 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "CR Beer".upper(), 'Advertiser_parent_standarized'] = "CR Beer (Heineken Parent)" ## changed
df.loc[df['Advertiser_parent_standarized'] == "OTHERS", 'Advertiser_parent_standarized'] = "All Others" ## Added
##Colombia
# In 2005, Bavaria Brewery became a subsidiary of SABMiller. SABMiller is subsidiary of ABI
# df.loc[df['Advertiser_parent_standarized'] == "BAVARIA", 'Advertiser_parent_standarized'] = "ABI" ## QURIED the market
df.loc[(df['Country'] == 'Colombia') & (df['Advertiser_parent_standarized'] == "BAVARIA"), 
       'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ABINBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CCC", 'Advertiser_parent_standarized'] = "CCC (Heineken Parent)" ## Chnaged
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN INT", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCA INT", 'Advertiser_parent_standarized'] = "CCA"
#'DOMINICAN REPUBLIC':
#df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA NACIONAL DOMINICANA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "UNITED BRANDS", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCU CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)" # Chnaged
df.loc[df['Advertiser_parent_standarized'] == "BUDWEISER CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CLUB COLOMBIA CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "QUILMES CABLE PANREGIONALL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MIKES HARD CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "AGUILA CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
#'ECUADOR'
df.loc[df['Advertiser_parent_standarized'] == "CIA. DE CERVEZAS NACIONALES", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "STELLA ARTOIS", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "Heineken", 'Advertiser_parent_standarized'] = "HEINEKEN"
#'EL SALVADOR'
df.loc[df['Advertiser_parent_standarized'] == "INDUSTRIAS LA CONSTANCIA SA DE CV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "DISAL", 'Advertiser_parent_standarized'] = "HEINEKEN"
#'GUATEMALA'
df.loc[df['Advertiser_parent_standarized'] == "AMBEV CENTROAMERICA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GRUPO MODELO CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MIKES HARD LEMONADE CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA CENTRO AMERICANA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCU CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"
#'HONDURAS'
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA HONDUREÑA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "DISTRIBUIDORA ISTMANIA", 'Advertiser_parent_standarized'] = "HEINEKEN"
#'INDIA'
df.loc[df['Advertiser_parent_standarized'] == "Pernod Ricard india private limited", 'Advertiser_parent_standarized'] = "Pernod Ricard".upper()
df.loc[df['Advertiser_parent_standarized'] == "bacardi martini india", 'Advertiser_parent_standarized'] = "BACARDI"
df.loc[df['Advertiser_parent_standarized'] == "CROWN BEERS INDIA LTD", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "Carlsberg Beer".upper(), 'Advertiser_parent_standarized'] = "Carlsberg".upper()
df.loc[df['Advertiser_parent_standarized'] == "united spirits limited", 'Advertiser_parent_standarized'] = "United Spirits".upper()
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN N.V.", 'Advertiser_parent_standarized'] = "HEINEKEN"

df.loc[df['Advertiser_parent_standarized'] == "BACARDI CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "BACARDI"
df.loc[df['Advertiser_parent_standarized'] == "BACARDI LIMITED", 'Advertiser_parent_standarized'] = "BACARDI"
df.loc[df['Advertiser_parent_standarized'] == "BACARDI LTD", 'Advertiser_parent_standarized'] = "BACARDI"
#'JAPAN' - na
#'MEXICO' -na
#'PARAGUAY'
df.loc[df['Advertiser_parent_standarized'] == "Bebidas del Paraguay", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "Distribuidora Gloria", 'Advertiser_parent_standarized'] = "MOLSON COORS"
#'PANAMA' ## is FEDURO should be Molson Coors - Ask Bob 
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA PANAMA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "DIST COMERCIAL GROUP", 'Advertiser_parent_standarized'] = "ABI"
#'PERU'
df.loc[df['Advertiser_parent_standarized'] == "UCP BACKUS Y JOHNSTON S.A.A.", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "LIMBA SERVICES SAC", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "DESTILERIA PERUANA", 'Advertiser_parent_standarized'] = "BACARDI"
#'SOUTH AFRICA'
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "SAB MILLER", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN INTERNATIONAL", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN SOUTH AFRICA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA MODELO", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "BELGIUM BEER COMPANY", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER BUSCH INC", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CORONA CHASING SUNSETS", 'Advertiser_parent_standarized'] = "ABI"
#'SOUTH KOREA'
df.loc[df['Advertiser_parent_standarized'] == "ABInBev", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GOLDEN BLUE", 'Advertiser_parent_standarized'] = "GOLDEN BLUE (MOLSON COORS)"
#'URUGUAY'
df.loc[df['Advertiser_parent_standarized'] == "FNC", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "FABRICAS NACIONALES DE CERVEZA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CORONA", 'Advertiser_parent_standarized'] = "ABI"
#df.loc[df['Advertiser_parent_standarized'] == "PILSEN", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "COMPANIA CERVECERIAS UNIDAS", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCU", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"
#'VIETNAM'
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG VIETNAM", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "VBL", 'Advertiser_parent_standarized'] = "VBL (Heineken Parent)"
# USA
df.loc[df['Advertiser_parent_standarized'] == "Anheuser-Busch InBev", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == 'MOLSON COORS BREWING CO', 'Advertiser_parent_standarized'] = "MOLSON COORS"
df.loc[df['Advertiser_parent_standarized'] == 'MOLSON', 'Advertiser_parent_standarized'] = "MOLSON COORS"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN NV", 'Advertiser_parent_standarized'] = "HEINEKEN"
#Bolivia
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA BOLIVIANA NACIONAL S.A.", 'Advertiser_parent_standarized'] = "ABI"
#df["Advertiser_parent_standarized"].unique()
##UK
df.loc[df['Advertiser_parent_standarized'] == "MOLSON COORS BREWING CO UK LTD", 'Advertiser_parent_standarized'] = "MOLSON COORS"
df.loc[df['Advertiser_parent_standarized'] == "DIAGEO UK LTD", 'Advertiser_parent_standarized'] = "DIAGEO"
df.loc[df['Advertiser_parent_standarized'] == "BROWN FORMAN BEVERAGES WORLDWIDE/COCA COLA COMPANY", 'Advertiser_parent_standarized'] = "BROWN FORMAN"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN UK LTD", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "ASAHI UK LTD", 'Advertiser_parent_standarized'] = "ASAHI"
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG UK LTD", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "AB INBEV UK LTD", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ASAHI BRANDS GERMANY, KOELN", 'Advertiser_parent_standarized'] = "ASAHI"
df.loc[df['Advertiser_parent_standarized'] == "ASAHI BREWERIES LTD", 'Advertiser_parent_standarized'] = "ASAHI"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA BRASSERIE", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA BRAU PTY LTD", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA ITALIA SRL TORINO", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA LIESHOUT", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "CONSTELLATION BRANDS INC", 'Advertiser_parent_standarized'] = "CONSTELLATION"
#Belgium
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER BUSCH INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "DUVEL MOORTGAT BROUWERIJ", 'Advertiser_parent_standarized'] = "DUVEL"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN COMPANY", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG IMPORTERS", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "MARTIN'S GROUP", 'Advertiser_parent_standarized'] = "MARTINS"

# France:
df.loc[df['Advertiser_parent_standarized'] == "AB INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA BRASSERIE", 'Advertiser_parent_standarized'] = "BAVARIA"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN FRANCE GPE", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG GROUPE", 'Advertiser_parent_standarized'] = "CARLSBERG"

#Germany
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN DT., BER", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG DT., HH", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH INBEV GERM.HOL., BREMEN", 
       'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "WARSTEINER BRAUEREI HS.CRAMER, WARSTEIN", 
       'Advertiser_parent_standarized'] = "WARSTEINER"
df.loc[df['Advertiser_parent_standarized'] == "KROMBACHER BRAUEREI, KREUZTAL-KROMBACH", 
       'Advertiser_parent_standarized'] = "KROMBACHER"
df.loc[df['Advertiser_parent_standarized'] == "PAULANER BRAUEREI, MUC", 
       'Advertiser_parent_standarized'] = "PAULANER"
df.loc[df['Advertiser_parent_standarized'] == "VELTINS C.+.A. BRAUEREI, MESCHEDE", 
       'Advertiser_parent_standarized'] = "VELTINS"
df.loc[df['Advertiser_parent_standarized'] == "FRIESISCHES BRAUHAUS ZU JEVER, JEVER", 
       'Advertiser_parent_standarized'] = "FRIESISCHES"
df.loc[df['Advertiser_parent_standarized'] == "RADEBERGER EXPORTBIERBRAUEREI, RADEBERG", 
       'Advertiser_parent_standarized'] = "RADEBERGER"

#Italy
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN ITALIA SPA MILANO", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG IT.INDUNO O.VA", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "AB INBEV IT.SPA GALLARATE VA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA ITALIA SRL TORINO", 'Advertiser_parent_standarized'] = "BAVARIA"

# Nl
df.loc[df['Advertiser_parent_standarized'] == "Heineken Nederland (WHK) Zoeterwoude".upper(), 
       'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "Koninklijke Grolsch Enschede.upper()", 'Advertiser_parent_standarized'] = "GROLSCH"
df.loc[df['Advertiser_parent_standarized'] == "Duvel Moortgat Nederland Amsterdam".upper(), 
       'Advertiser_parent_standarized'] = "DUVEL"
df.loc[df['Advertiser_parent_standarized'] == "AB InBev Nederland Breda".upper(), 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "TEXELSE BIERBROUWERIJ OUDESCHILD", 'Advertiser_parent_standarized'] = "TEXELSE"
df.loc[df['Advertiser_parent_standarized'] == "BROUWERIJ BOSTEELS BUGGENHOUT [BE]", 
       'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "LIDL NEDERLAND HUIZEN", 'Advertiser_parent_standarized'] = "HUIZEN"
df.loc[df['Advertiser_parent_standarized'] == "CAMDEN TOWN BREWERY LTD", 'Advertiser_parent_standarized'] = "ABI"


In [17]:
#df['Advertiser_parent_standarized'].unique()

### Brand (standardized) 

In [18]:
df["Brand_standarization"] = df["Brand"].copy() # Copy the brnad names to a new column to strd
df["Brand_standarization"] = df["Brand_standarization"].str.strip()
df["Brand_standarization"] = df["Brand_standarization"].str.replace('SPONSORSHIP', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('CORPORATE', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('COMPETITION', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('RUGBY', '')
#df["Brand_standarization"] = df["Brand_standarization"].str.replace('(REG)', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('(BEER)', '')
#df["Brand_standarization"] = df["Brand_standarization"].str.replace('(CERVEZA)', '')
#df["Brand_standarization"] = df["Brand_standarization"].str.replace("(CERVECERIA)", "")
#df["Brand_standarization"] = df["Brand_standarization"].str.replace("(", "")
#df["Brand_standarization"] = df["Brand_standarization"].str.replace(")", "")
df["Brand_standarization"] = df["Brand_standarization"].str.strip()
df.loc[df['Brand_standarization'] == "HEINEKEN 0 0", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN O.O LAGER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN 0 0 A1PADEL PAYSPACE CAPE TOWN MASTER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN O.O LAGER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN 00", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN MEXICO", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN LAGER", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN ()", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == 'UNITED BRANDS', 'Brand_standarization'] = "HEINEKEN" #Dominican Republic
df.loc[df['Brand_standarization'] == "GC/AMSTEL MALTA", 'Brand_standarization'] = "AMSTEL MALTA"

df.loc[df['Brand_standarization'] == "HEINEKEN - EURO 2020", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN - CHAMPIONS LEAGUE", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN - UEFA CHAMPIONS LEAGUE", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "BIRRA MORETTI - ZERO BEER", 'Brand_standarization'] = "BIRRA MORETTI 0.0"
df.loc[df['Brand_standarization'] == "BIRRA MORETTI - BRAND BUILDING", 'Brand_standarization'] = "BIRRA MORETTI"
df.loc[df['Brand_standarization'] == "HEINEKEN - HEINEKEN SILVER", 'Brand_standarization'] = "HEINEKEN SILVER"
df.loc[df['Brand_standarization'] == "HEINEKEN - UEFA EUROPA LEAGUE SPONSOR", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN/TURKISH AIRLINES - UEFA CHAMPIONS LEAGUE", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN - BRAND BUILDING", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN - UEFA CHAMPIONS LEAGUE SPONSOR", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN/OPPO - UEFA CHAMPIONS LEAGUE", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN - UEFA WOMENS CHAMPIONS LEAGUE SPONSOR", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "ASAHI UK LTD", 'Brand_standarization'] = "ASAHI"

df.loc[df['Brand_standarization'] == "CARLSBERG - 00 ALCOHOL FREE LAGER", 'Brand_standarization'] = "CARLSBERG"
df.loc[df['Brand_standarization'] == "CARLSBERG - LIVERPOOL FC SPONSOR", 'Brand_standarization'] = "CARLSBERG"
df.loc[df['Brand_standarization'] == "CARLSBERG - DANISH PILSNER", 'Brand_standarization'] = "CARLSBERG"
df.loc[df['Brand_standarization'] == "ASAHI UK LTD", 'Brand_standarization'] = "ASAHI"

df.loc[df['Brand_standarization'] == "CORONA EXTRA - LAGER", 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == "STELLA ARTOIS - UNFILTERED", 'Brand_standarization'] = "STELLA ARTOIS"
df.loc[df['Brand_standarization'] == "CORONA - BRAND BUILDING", 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == "CORONA - TROPICAL", 'Brand_standarization'] = "CORONA TROPICAL"
df.loc[df['Brand_standarization'] == "LEFFE - BLONDE BEER", 'Brand_standarization'] = "LEFFE"
df.loc[df['Brand_standarization'] == "CORONA - CERO", 'Brand_standarization'] = "CORONA CERO"
df.loc[df['Brand_standarization'] == "BUDWEISER BEER - BRAND BUILDING", 'Brand_standarization'] = "BUDWEISER"
df.loc[df['Brand_standarization'] == "BECKS - AUTONOMOUS", 'Brand_standarization'] = "BECKS"


In [19]:
## Other Brands including ABI
df.loc[df['Brand_standarization'] == "BRUGAL ()", 'Brand_standarization'] = "BRUGAL"
df.loc[df['Brand_standarization'] == "CORONA.", 'Brand_standarization'] = "CORONA" # 'ARGENTINA'
df.loc[df['Brand_standarization'] == 'PACEÑA', 'Brand_standarization'] = "PACENA" # Bolivia
df.loc[df['Brand_standarization'] == 'AUSTRAL (CERVEZA)', 'Brand_standarization'] = "AUSTRAL" #'CHILE'
df.loc[df['Brand_standarization'] == 'CRISTAL (CERVEZA)', 'Brand_standarization'] = "CRISTAL"
df.loc[df['Brand_standarization'] == 'PATAGONIA CERVEZA', 'Brand_standarization'] = "PATAGONIA"
df.loc[df['Brand_standarization'] == 'PATAGONIA (CERVEZA)', 'Brand_standarization'] = "PATAGONIA"
df.loc[df['Brand_standarization'] == 'SOL (CERVEZA)', 'Brand_standarization'] = "SOL"
df.loc[df['Brand_standarization'] == 'ABINBEV INSTITUCIONAL', 'Brand_standarization'] = "ABI"
df.loc[df['Brand_standarization'] == 'CORONA (CERVEZA)', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CUSQUEÑA', 'Brand_standarization'] = "CUSQUENA"
df.loc[df['Brand_standarization'] == "Snow Others", 'Brand_standarization'] = "SNOW"  #'CHINA'
df.loc[df['Brand_standarization'] == "Others", 'Brand_standarization'] = "All Others"
df.loc[df['Brand_standarization'] == "CORONITA", 'Brand_standarization'] = "CORONA"  #'COLOMBIA'
df.loc[df['Brand_standarization'] == "COSTEÑA", 'Brand_standarization'] = "COSTENA"
df.loc[df['Brand_standarization'] == 'CERVECERIA NACIONAL DOMINICANA', 'Brand_standarization'] = "CERVECERIA" #'Dom Rep'
df.loc[df['Brand_standarization'] == 'CIA. CERVEZAS NACIONALES', 'Brand_standarization'] = "ABI"  #'ECUADOR'
df.loc[df['Brand_standarization'] == 'CASTLE LITE BEER', 'Brand_standarization'] = "CASTLE LITE" #'SOUTH AFRICA'
df.loc[df['Brand_standarization'] == 'BRUTAL FRUIT SPRITZER', 'Brand_standarization'] = "BRUTAL FRUIT"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER SOCCER', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CASTLE LITE BEER', 'Brand_standarization'] = "CASTLE LITE"
df.loc[df['Brand_standarization'] == 'FLYING FISH CHILL BEER', 'Brand_standarization'] = "FLYING FISH"
#df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS', 'Brand_standarization'] = "FLYING FISH"
#df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS', 'Brand_standarization'] = "FLYING FISH FLAVOUR CHILLAS"
df.loc[df['Brand_standarization'] == 'FLYING FISH PREMIUM FLAVOURED BEER', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS', 'Brand_standarization'] = "FLYING FISH FLAVOUR CHILLAS"
df.loc[df['Brand_standarization'] == 'FLYING FISH PREMIUM FLAVOURED BEER COMP', 'Brand_standarization'] = "FLYING FISH"
#df.loc[df['Brand_standarization'] == 'FLYING FISH SELTZER', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH SELTZER PROMOTION', 'Brand_standarization'] = "FLYING FISH SELTZER"
df.loc[df['Brand_standarization'] == 'HUNTERS CIDER COMPETITION', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS CIDER RANGE', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS DRY CIDER', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS REFRESH THE RULES OF IGRAND', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'SAVANNA CIDER MUSIC', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA CIDER RANGE', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA COMICS AWARDS', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA DRY CIDER', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA DRY CIDER COMEDY', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'HEINEKEN LAGER BEER', 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN LAGER BEER RUGBY', 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY COMPETITION', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY PROMOTION', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER BLACK LABEL', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER FLAVOURCODE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER GOLD LABEL RESERVE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER WHISKY RANGE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'CORONA SUNSET FESTIVAL', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA OPEN JBAY', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'BLACK CROWN PREMIUM GIN & TONIC', 'Brand_standarization'] = "BLACK CROWN"
df.loc[df['Brand_standarization'] == 'STELLA ARTOIS BEER', 'Brand_standarization'] = "STELLA ARTOIS"
df.loc[df['Brand_standarization'] == 'STELLA ARTOIS PLAYERS CHAMPIONSHIP', 'Brand_standarization'] = "STELLA ARTOIS"
df.loc[df['Brand_standarization'] == 'BUDWEISER BEER', 'Brand_standarization'] = "BUDWEISER"
df.loc[df['Brand_standarization'] == 'CORONA CHASING SUNSETS', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CASTLE MILK STOUT MEDLEY PROMOTION', 'Brand_standarization'] = "CASTLE MILK STOUT"
df.loc[df['Brand_standarization'] == 'Bud Light Corporate'.upper(), 'Brand_standarization'] = "Bud Light".upper() #USA 
df.loc[df['Brand_standarization'] == 'Bud Light Seltzer Corporate'.upper(), 'Brand_standarization'] = "Bud Light Seltzer".upper()
df.loc[df['Brand_standarization'] == 'Budweiser Corporate'.upper(), 'Brand_standarization'] = "Budweiser".upper()
#df.loc[df['Brand_standarization'] == 'Corona Extra'.upper(), 'Brand_standarization'] = "Corona".upper()
#df.loc[df['Brand_standarization'] == 'Corona Extra Corporate'.upper(), 'Brand_standarization'] = "Corona".upper()
df.loc[df['Brand_standarization'] == 'Dos Equis Corporate'.upper(), 'Brand_standarization'] = "Dos Equis".upper()
df.loc[df['Brand_standarization'] == 'Michelob Ultra Corporate'.upper(), 'Brand_standarization'] = "Michelob Ultra".upper()
df.loc[df['Brand_standarization'] == 'Miller Lite Corporate'.upper(), 'Brand_standarization'] = "Miller Lite".upper()
df.loc[df['Brand_standarization'] == 'Modelo Corporate'.upper(), 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'Anheuser-Busch'.upper(), 'Brand_standarization'] = "ABI"
df.loc[df['Brand_standarization'] == 'Anheuser-Busch Corporate'.upper(), 'Brand_standarization'] = "ABI"                                         
df.loc[df['Brand_standarization'] == 'Budweiser Sponsorship'.upper(), 'Brand_standarization'] =  "Budweiser".upper()                                          
df.loc[df['Brand_standarization'] == 'Bud Light Sponsorship'.upper(), 'Brand_standarization'] =  "Bud Light".upper()              
df.loc[df['Brand_standarization'] == 'Cerveza Victoria'.upper(), 'Brand_standarization'] =  "Victoria".upper() 
# Ad-hoc chnages:
df.loc[df['Brand_standarization'] == 'TIGER ()', 'Brand_standarization'] =  "TIGER"
df.loc[df['Brand_standarization'] == 'HUDA ()', 'Brand_standarization'] =  "HUDA"
df.loc[df['Brand_standarization'] ==  'KRONENBOURG 1664 BLANC ()', 'Brand_standarization'] =  "KRONENBOURG 1664 BLANC"
df.loc[df['Brand_standarization'] == 'TUBORG ()', 'Brand_standarization'] =  "TUBORG"
df.loc[df['Brand_standarization'] == 'CRISTAL ()', 'Brand_standarization'] =  "CRISTAL"
#df.loc[df['Brand_standarization'] ==   'IMPERIAL ()', 'Brand_standarization'] =  "IMPERIAL"
df.loc[df['Brand_standarization'] ==  'SOL (CERVEZA)', 'Brand_standarization'] =  "SOL"
df.loc[df['Brand_standarization'] == 'BUDWEISER ()', 'Brand_standarization'] =  "BUDWEISER"
df.loc[df['Brand_standarization'] == 'CRISTAL (CERVEZA)', 'Brand_standarization'] =  "CRISTAL"
df.loc[df['Brand_standarization'] == 'IMPERIAL (CERVEZA)', 'Brand_standarization'] =  "IMPERIAL"
df.loc[df['Brand_standarization'] == 'LOA,CERVEZA', 'Brand_standarization'] =  "LOA"
df.loc[df['Brand_standarization'] == 'PATAGONIA CERVEZA', 'Brand_standarization'] =  "PATAGONIA"
df.loc[df['Brand_standarization'] == 'AUSTRAL (CERVEZA)', 'Brand_standarization'] =  "AUSTRAL"
df.loc[df['Brand_standarization'] == 'CORONA (CERVEZA)', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'EDELWEISS (CERVEZA)', 'Brand_standarization'] =  "EDELWEISS"
df.loc[df['Brand_standarization'] == 'MILLER (CERVEZA)', 'Brand_standarization'] =  "MILLER"
df.loc[df['Brand_standarization'] == 'PATAGONIA (CERVEZA)', 'Brand_standarization'] =  "PATAGONIA"
df.loc[df['Brand_standarization'] == 'CORONA (CERVEZA)', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CERVEZA BAJAMAR', 'Brand_standarization'] =  "BAJAMAR"
df.loc[df['Brand_standarization'] == 'CERVEZA CAMINO DEL SOL', 'Brand_standarization'] =  "CAMINO DEL SOL"
df.loc[df['Brand_standarization'] == 'CIA. CERVEZAS NACIONALES', 'Brand_standarization'] =  "CIA. NACIONALES"
df.loc[df['Brand_standarization'] == 'CERVEZA DORADA DRAFT', 'Brand_standarization'] =  "DORADA DRAFT"
df.loc[df['Brand_standarization'] == 'CERVEZA DORADA ICE', 'Brand_standarization'] =  "DORADA ICE"
df.loc[df['Brand_standarization'] == 'CERVEZA GALLO', 'Brand_standarization'] =  "GALLO"
df.loc[df['Brand_standarization'] == 'GALLO', 'Brand_standarization'] =  "GALLO"
df.loc[df['Brand_standarization'] == 'CERVEZA TECATE', 'Brand_standarization'] =  "TECATE"
df.loc[df['Brand_standarization'] == 'CERVEZA BARENA', 'Brand_standarization'] =  "BARENA"
df.loc[df['Brand_standarization'] == 'CERVEZA CORONA', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CERVEZA GROLSCH', 'Brand_standarization'] =  "GROLSCH"
df.loc[df['Brand_standarization'] == 'CERVEZA HEINEKEN', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'CERVEZA MICHELOB', 'Brand_standarization'] =  "MICHELOB"
df.loc[df['Brand_standarization'] == 'CERVEZA MILLER', 'Brand_standarization'] =  "MILLER"
df.loc[df['Brand_standarization'] == 'CERVEZA SALVA VIDA', 'Brand_standarization'] =  "SALVA VIDA"
df.loc[df['Brand_standarization'] == 'BALBOA CERVEZA', 'Brand_standarization'] =  "BALBOA"
df.loc[df['Brand_standarization'] == 'CERVEZA DORADA', 'Brand_standarization'] =  "DORADA"
df.loc[df['Brand_standarization'] == 'CERVEZA CRISTAL', 'Brand_standarization'] =  "CRISTAL"
df.loc[df['Brand_standarization'] == 'GALLO CERVEZA', 'Brand_standarization'] =  "GALLO"
df.loc[df['Brand_standarization'] == 'CERVEZA IMPERIAL', 'Brand_standarization'] =  "IMPERIAL"
df.loc[df['Brand_standarization'] == 'CERVEZA PALMA', 'Brand_standarization'] =  "PALMA"
df.loc[df['Brand_standarization'] == 'PATAGONIA CERVEZA', 'Brand_standarization'] =  "PATAGONIA"
df.loc[df['Brand_standarization'] == 'PRESIDENTE CERVEZA', 'Brand_standarization'] =  "PRESIDENTE"
df.loc[df['Brand_standarization'] == 'CERVEZA RREY', 'Brand_standarization'] =  "RREY"
df.loc[df['Brand_standarization'] == 'CERVEZA SALTA CAUTIVA', 'Brand_standarization'] =  "SALTA CAUTIVA"
df.loc[df['Brand_standarization'] == 'CERVEZA VICTORIA', 'Brand_standarization'] =  "VICTORIA"
df.loc[df['Brand_standarization'] == 'BIERE LARUE ()', 'Brand_standarization'] =  "LARUE"
df.loc[df['Brand_standarization'] == 'HANOI ()', 'Brand_standarization'] =  "HANOI"
df.loc[df['Brand_standarization'] == 'TRUC BACH ()', 'Brand_standarization'] =  "TRUC BACH"
df.loc[df['Brand_standarization'] == 'SAO VANG ()', 'Brand_standarization'] =  "SAO VANG"
df.loc[df['Brand_standarization'] == 'BUDWEISER (REG)', 'Brand_standarization'] =  "BUDWEISER"
df.loc[df['Brand_standarization'] == 'CORONA (REG)', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'HEINEKEN (REG)', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER SOCCER', 'Brand_standarization'] = "CARLING BLACK LABEL" 

df.loc[df['Brand_standarization'] == 'GUINNESS - BRAND BUILDING', 'Brand_standarization'] = "GUINNESS"
df.loc[df['Brand_standarization'] == 'BREWDOG - PUNK IPA', 'Brand_standarization'] = "BREWDOG" 
df.loc[df['Brand_standarization'] == 'TIMOTHY TAYLOR - HOPICAL STORM PALE', 'Brand_standarization'] = "TIMOTHY TAYLOR" 
df.loc[df['Brand_standarization'] == 'BEAVERTOWN BREWERY - CRAFT S & LAGER' , 'Brand_standarization'] = "BEAVERTOWN BREWERY"
df.loc[df['Brand_standarization'] == 'ASAHI - SUPER DRY', 'Brand_standarization'] = "ASAHI"
df.loc[df['Brand_standarization'] == 'COORS - FRESH COMEDY ON 4', 'Brand_standarization'] = "COORS" 
df.loc[df['Brand_standarization'] == 'MOLSON COORS BREWING - MADRI EXCEPCIONAL', 'Brand_standarization'] = "MOLSON COORS" 
df.loc[df['Brand_standarization'] == 'JACK DANIELS/COCA COLA - TENNESSEE WHISKEY MIXED WITH COCA COLA', 
       'Brand_standarization'] = "JACK DANIELS"

## MODELO categorization for US, Mexico, DR, Guatemala
df.loc[df['Brand_standarization'] == 'MODELO ESPECIAL', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELO ORO', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELO NEGRA', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELO NOCHE ESPECIAL', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELO LIGHT', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELO ESPECIAL CHELADA', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'GRUPO MODELO', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'MODELORAMA', 'Brand_standarization'] = "MODELO"
df.loc[df['Brand_standarization'] == 'CERVEZA MODELO', 'Brand_standarization'] = "MODELO"

# Ad-hoc updates
df.loc[df['Brand_standarization'] == 'CORONA PREMIER', 'Brand_standarization'] =  "CORONA"
#df.loc[df['Brand_standarization'] == 'CORONA PREMIER', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'XX', 'Brand_standarization'] =  "DOS EQUIS"
df.loc[df['Brand_standarization'] == 'DOS EQUIS XX BIERE', 'Brand_standarization'] =  "DOS EQUIS"
df.loc[df['Brand_standarization'] == 'MIKE S HARD', 'Brand_standarization'] =  "MIKES"
df.loc[df['Brand_standarization'] == 'MIKES HARD', 'Brand_standarization'] =  "MIKES"
df.loc[df['Brand_standarization'] == 'MIKES (REG)', 'Brand_standarization'] =  "MIKES"
df.loc[df['Brand_standarization'] == 'MIKES HARDER LEMONADE', 'Brand_standarization'] =  "MIKES"
df.loc[df['Brand_standarization'] == 'MIKES HARDER', 'Brand_standarization'] =  "MIKES"
df.loc[df['Brand_standarization'] == 'AB INBEV', 'Brand_standarization'] =  "ABI"
df.loc[df['Brand_standarization'] == 'BELGIUM', 'Brand_standarization'] =  "ABI"  ## What it is
df.loc[df['Brand_standarization'] == 'HEINEKEN SILVER BIERE', 'Brand_standarization'] =  "HEINEKEN SILVER"
df.loc[df['Brand_standarization'] == 'ABBAYE DE LEFFE BIERE', 'Brand_standarization'] =  "LEFFE"
df.loc[df['Brand_standarization'] == 'BUD BIERE', 'Brand_standarization'] =  "BUDWEISER"
df.loc[df['Brand_standarization'] == 'CORONA EXTRA BIERE', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA SUNSET BIERE', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA EXTRA', 'Brand_standarization'] =  "CORONA" ##
df.loc[df['Brand_standarization'] == 'CORONA SUNSET BIERE', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'LEFFE BIERES', 'Brand_standarization'] =  "LEFFE"
df.loc[df['Brand_standarization'] == 'HEINEKEN BIERE', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'AMSTEL BIERE COLLECTIVITE', 'Brand_standarization'] =  "AMSTEL"
df.loc[df['Brand_standarization'] == '1664 BLONDE', 'Brand_standarization'] =  "#1664"
df.loc[df['Brand_standarization'] == '8 6 BIERE', 'Brand_standarization'] =  "#86"
#df.loc[df['Brand_standarization'] == 'GALLIA PARIS BIERES', 'Brand_standarization'] =  "GALLIA"
df.loc[df['Brand_standarization'] == 'TOURTEL TWIST FRUITS & PLANTES SANS ALCOOL ALLEGE', 
       'Brand_standarization'] =  "TOURTEL"
df.loc[df['Brand_standarization'] == 'TOURTEL TWIST BIERE SANS ALCOOL', 'Brand_standarization'] =  "TOURTEL"
#df.loc[df['Brand_standarization'] == 'DESPERADOS TROPICAL BIERE', 'Brand_standarization'] =  "DESPERADOS TROPICAL"
df.loc[df['Brand_standarization'] == 'AFFLIGEM BIERE', 'Brand_standarization'] =  "AFFLIGEM"
df.loc[df['Brand_standarization'] == 'GRIMBERGEN BRASSIN DE PRINTEMPS', 'Brand_standarization'] =  "GRIMBERGEN"

#df.loc[df['Brand_standarization'] == 'DESPERADOS 0,0 PROZ.ALKOHOLFREI', 'Brand_standarization'] =  "DESPERADOS 0.0"
#df.loc[df['Brand_standarization'] == 'DESPERADOS BIER RANGE', 'Brand_standarization'] =  "DESPERADOS RANGE"
#df.loc[df['Brand_standarization'] == 'DESPERADOS SPECIAALBIER', 'Brand_standarization'] =  "DESPERADOS"
#df.loc[df['Brand_standarization'] == 'DESPERADOS VIRGIN 0.0 ALCOHOLVRIJ SPECIAALBIER', 
#       'Brand_standarization'] =  "DESPERADOS VIRGIN 0.0"

df.loc[df['Brand_standarization'] == 'HEINEKEN BIER', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN BIER 0,0 PROZENT', 'Brand_standarization'] =  "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == 'HEINEKEN BIER RANGE', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'CARLSBERG 0,0 ALKOHOLFREI', 'Brand_standarization'] =  "CARLSBERG 0.0"
df.loc[df['Brand_standarization'] == 'CARLSBERG BEER IMAGE', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'CARLSBERG BEER PILS', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'CARLSBERG BRAUEREI BIER RANGE', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'CARLSBERG PREMIUM LAGER BIER', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'HEINEKEN 0.0 BIRRA ANALC.', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN BIRRA', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN LIN.BIRRA', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN SILVER BIRRA', 'Brand_standarization'] =  "HEINEKEN SILVER"
df.loc[df['Brand_standarization'] == 'ICHNUSA BIRRA', 'Brand_standarization'] =  "ICHNUSA"
df.loc[df['Brand_standarization'] == 'ICHNUSA BIRRA I.U.', 'Brand_standarization'] =  "ICHNUSA"
df.loc[df['Brand_standarization'] == 'MESSINA BIRRA', 'Brand_standarization'] =  "MESSINA"
df.loc[df['Brand_standarization'] == 'MORETTI BIRRA', 'Brand_standarization'] =  "MORETTI"
df.loc[df['Brand_standarization'] == 'MORETTI BIRRA WWW', 'Brand_standarization'] =  "MORETTI"
df.loc[df['Brand_standarization'] == 'MORETTI FILTRATA FREDDO BIRRA', 'Brand_standarization'] =  "MORETTI"
df.loc[df['Brand_standarization'] == 'MORETTI LIN.BIRRA', 'Brand_standarization'] =  "MORETTI"
df.loc[df['Brand_standarization'] == 'PERONI NASTRO AZZ.STILE CAPRI', 'Brand_standarization'] =  "PERONI NASTRO"
df.loc[df['Brand_standarization'] == 'PERONI NASTRO AZZURRO BIRRA', 'Brand_standarization'] =  "PERONI NASTRO"
df.loc[df['Brand_standarization'] == 'KOZEL BIRRA', 'Brand_standarization'] =  "KOZEL"
df.loc[df['Brand_standarization'] == 'CARLSBERG (T)', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'CARLSBERG BIRRA', 'Brand_standarization'] =  "CARLSBERG"
df.loc[df['Brand_standarization'] == 'GRIMBERGEN BIRRA', 'Brand_standarization'] =  "GRIMBERGEN"
df.loc[df['Brand_standarization'] == 'PORETTI 4 LUPPOLI BIRRA', 'Brand_standarization'] =  "GRIMBERGEN"
df.loc[df['Brand_standarization'] == 'PORETTI BIRRA', 'Brand_standarization'] =  "GRIMBERGEN"
df.loc[df['Brand_standarization'] == 'PORETTI LIN.BIRRA', 'Brand_standarization'] =  "GRIMBERGEN"
df.loc[df['Brand_standarization'] == 'CERES BIRRA', 'Brand_standarization'] =  "CERES"
df.loc[df['Brand_standarization'] == 'CERES PRO.BIRRA', 'Brand_standarization'] =  "CERES"

df.loc[df['Brand_standarization'] == "BECK'S BIRRA", 'Brand_standarization'] =  "BECKS"
df.loc[df['Brand_standarization'] == 'CORONA BIRRA', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA BIER RANGE', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA BIER RANGE              VB', 'Brand_standarization'] =  "CORONA"

df.loc[df['Brand_standarization'] == 'CORONA EXTRA BIRRA', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA SUNSETS FESTIVAL EVENTI', 'Brand_standarization'] =  "CORONA"
df.loc[df['Brand_standarization'] == 'LEFFE BIRRA', 'Brand_standarization'] =  "LEFFE"
df.loc[df['Brand_standarization'] == 'LEFFE CO.BIRRA', 'Brand_standarization'] =  "LEFFE"

df.loc[df['Brand_standarization'] == 'AFFLIGEM BLOND 0.0% ALCOHOLVRIJ SPECIAALBIER', 'Brand_standarization'] =  "AFFLIGEM 0.0"
df.loc[df['Brand_standarization'] == 'AFFLIGEM BLOND SPECIAALBIER', 'Brand_standarization'] =  "AFFLIGEM"
df.loc[df['Brand_standarization'] == 'AFFLIGEM SPECIAALBIER ALGEMEEN', 'Brand_standarization'] =  "AFFLIGEM"

df.loc[df['Brand_standarization'] == 'AMSTEL 0.0 ALCOHOLVRIJ BIER', 'Brand_standarization'] =  "AMSTEL 0.0"
df.loc[df['Brand_standarization'] == 'AMSTEL BIER PILS', 'Brand_standarization'] =  "AMSTEL"
df.loc[df['Brand_standarization'] == 'AMSTEL RADLER', 'Brand_standarization'] =  "AMSTEL"
df.loc[df['Brand_standarization'] == 'AMSTEL RADLER 0.0% ALCOHOLVRIJ BIER', 'Brand_standarization'] =  "AMSTEL 0.0"

df.loc[df['Brand_standarization'] == 'BIRRA MORETTI BIER PILS', 'Brand_standarization'] =  "MORETTI"
df.loc[df['Brand_standarization'] == 'BIRRA MORETTI SALE DI MARE SPECIAALBIER', 'Brand_standarization'] =  "MORETTI"

df.loc[df['Brand_standarization'] == 'HEINEKEN (ADV.) PRODUCTEN ALGEMEEN', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN 0.0 ALCOHOLVRIJ BIER', 'Brand_standarization'] =  "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == 'HEINEKEN BIER PILS', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN PROMOTIE', 'Brand_standarization'] =  "HEINEKEN"
df.loc[df['Brand_standarization'] == 'HEINEKEN SILVER BIER PILS', 'Brand_standarization'] =  "HEINEKEN SILVER"

df.loc[df['Brand_standarization'] == 'BUD BIER PILS', 'Brand_standarization'] =  "BUDWEISER"
df.loc[df['Brand_standarization'] == 'BUDWEISER BUDVAR BIER PILS', 'Brand_standarization'] =  "BUDWEISER"
df.loc[df['Brand_standarization'] == 'HERTOG JAN BIER PILS', 'Brand_standarization'] =  "HERTOG JAN"
df.loc[df['Brand_standarization'] == 'HERTOG JAN BOCKBIER SEIZOENSBIER', 'Brand_standarization'] =  "HERTOG JAN"
df.loc[df['Brand_standarization'] == 'HERTOG JAN GRAND PILSENER BIER PILS', 'Brand_standarization'] =  "HERTOG JAN"

df.loc[df['Brand_standarization'] == 'TRIPEL KARMELIET SPECIAALBIER', 'Brand_standarization'] =  "TRIPEL KARMELIET"
df.loc[df['Brand_standarization'] == 'KORDAAT PILSENER BIER', 'Brand_standarization'] =  "PILSENER"
df.loc[df['Brand_standarization'] == 'DUVEL SPECIAALBIEREN', 'Brand_standarization'] =  "DUVEL"
df.loc[df['Brand_standarization'] == 'KONINKLIJKE GROLSCH ENSCHEDE', 'Brand_standarization'] =  "GROLSCH"

df.loc[df['Brand_standarization'] == 'OTHERS', 'Brand_standarization'] =  "All Others"



In [20]:
#Code Test
#brands = df['Brand_standarization'].value_counts().to_frame()
#brands = brands.reset_index()

### Focused Brand Competitors

In [21]:
## Focus brand for the market:  It woudl be constant throughout the year

focus_brands = ['HEINEKEN', "HEINEKEN SILVER", "HEINEKEN 0.0", "AMSTEL", "AMSTEL ULTRA", 
               "AMSTEL ULTRA SELTZER", "AMSTEL LAGER", "AMSTEL MALTA", "AMSTEL LIGHT"]

df["Focused_Brand_Competitors"] = df["Brand_standarization"].copy()

# Assign most values in the 'Top_Brand_Competitors_and_ABI' column as 'Other' except for specific values
df['Focused_Brand_Competitors'] = np.where(df['Focused_Brand_Competitors'].isin(focus_brands), 
                                              df['Focused_Brand_Competitors'], 'All Others')
df['Focused_Brand_Competitors'].unique()

array(['All Others', 'HEINEKEN', 'AMSTEL', 'AMSTEL ULTRA', 'HEINEKEN 0.0',
       'HEINEKEN SILVER', 'AMSTEL ULTRA SELTZER', 'AMSTEL LAGER',
       'AMSTEL MALTA', 'AMSTEL LIGHT'], dtype=object)

### Media Summary

In [22]:
#df.Media_type.unique()

In [23]:
# Standarize Media Summary column
df["Media_Summary"] = df["Media_type"].copy() 

tv = ['Tv', 'Cable TV', 'PAY TV', 'Span Lang Net TV', 'TVPagada', 'CABLE', 'LOCAL TV', 'Network TV', 'Open TV', 'OPEN TV', 
      'OTV', 'Television', 'Advertising Broadcasts', 'PAID TV (CABLE)',
       'PTV', 'Spot TV', 'TELEVISION', 'TV', 'tv', 'TV ABIERTA', 'TV Abierta', 'TV ABIERTA ', 'TV CABLE',
      'TV Capital', 'TV Interior', 'TV PAGADA', 'TV Cable', 'Syndication']
radio = ['Natl Spot Radio', 'Radio AM', 'Local Radio', 'Network Radio', 'RADIO', 'Radio', 'Radio FM', 'Radios', 'RD']
print_ = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'PRESS', 'REVISTAS', 'B-to-B Magazines', 'DIARIOS', 
          'Diarios Interior', 'Hispanic Newspapers', 'Local Magazines', 'Local Media', 'MAGAZINE', 'Magazine', 'Magazines',
          'METRO', 'Newspaper', 'NP', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT', 'Revista', 'Revistas', 'Suplemento',
         'Press', 'Door Drops', 'PRESSE', 'Trade Journals', 'Trade press']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública', 'Cinema', 'CINEMA', 'VP', '(D)OOH', 
      'Poster', 'Transport Media', 'OutOfHome', 'Out Of Home', 'Ambient Media']
digital = ['INTERNET', 'Internet', 'Internet - Display', 'Internet - Search', 'Digital', 'Programmatic', 
           'DIGITAL', 'META', 'Mobile App', 'Mobile Web', 'Mobile Web Video', 'Online Video', 'Paid Social', 'Social',
           'Youtube', 'AVOD', 'Search', 'IN-HOME ADVERTISING', 'Internet Display Ag.', 'Mobile', 'Desktop', 'Retail Media',
          'DigitalMed', 'Online display']

df['Media_Summary'] = df['Media_Summary'].replace(tv, "TV")
df['Media_Summary'] = df['Media_Summary'].replace(radio, "Radio")
df['Media_Summary'] = df['Media_Summary'].replace(print_, "Print")
df['Media_Summary'] = df['Media_Summary'].replace(ooh, "OOH")
df['Media_Summary'] = df['Media_Summary'].replace(digital, "Digital")

df['Media_Summary'].unique()

array(['TV', 'Radio', 'Print', 'Digital', 'OOH'], dtype=object)

### Media Type (Standardized)

In [24]:
# Media Type (Standardized)
df["Media_type_standarized"] = df["Media_type"].copy() 

cable_tv = ['Cable TV', 'CABLE', 'PAID TV (CABLE)', 'TV CABLE', 'TV Cable']
open_tv = ['OPEN TV', 'OTV', 'Open TV']
paid_tv = ['PAY TV', 'TVPagada', 'PTV', 'TV PAGADA', 'PAID TV (CABLE)']
free_tv = ['TV ABIERTA', 'TV Abierta', 'TV ABIERTA ']
spot_tv = [ 'Spot TV']
tv = ['TELEVISION', 'TV', 'tv', 'Tv', 'Advertising Broadcasts', 'Television']
am_radio = ['Radio AM']
fm_radio = [ 'Radio FM']
local_radio = [ 'Local Radio']
natl_spot_radio = ['Natl Spot Radio']
network_radio = [ 'Network Radio']
radio = ['RADIO', 'Radio', 'Radios', 'RD']
cinema = ['Cinema', 'CINEMA']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública',  'VP', '(D)OOH', 'Poster', 
       'Transport Media', 'Ambient Media','Retail Media', 'OutOfHome', 'Out Of Home']
print_ = ['PRESS', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT', 'Press', 'PRESSE']
newspapers = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'Diarios Interior',
              'METRO', 'Newspaper', 'NP', 'Suplemento', 'DIARIOS', 'Trade press']
magazines = ['REVISTAS', 'MAGAZINE', 'Magazine', 'Magazines', 'Revista', 
             'Revistas', 'Door Drops']
digital = ['Digital', 'DIGITAL', 'Search', 'IN-HOME ADVERTISING', 'DigitalMed']
syndication = ['Syndication']


avod = ['AVOD']
internet = ['Internet', 'INTERNET']
internet_display = ['Internet - Display', 'Internet Display Ag.']
internet_search = ['Internet - Search']
meta = ['META']

mobile = ['Mobile']
desktop = ['Desktop']
mobile_app = [ 'Mobile App']    
mobile_web = ['Mobile Web']
mobile_web_video = ['Mobile Web Video']
online_video = ['Online Video']
paid_social = ['Paid Social']
programmatic = ['Programmatic']
social = ['Social']
yt = ['Youtube']
search = ['Search']
bb_magazines = ['B-to-B Magazines']
trade_magazines = ['Trade Journals']
hispanic_newspapers = ['Hispanic Newspapers']
local_magazines = ['Local Magazines'] 
local_media = ['Local Media']
local_tv = ['LOCAL TV'] 
span_lang_net_tv = ['Span Lang Net TV']
tv_capital = ['TV Capital']
tv_interior = ['TV Interior']

df['Media_type_standarized'] = df['Media_type_standarized'].replace(avod, "AVOD")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet, "Internet")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet_display, "Internet Display")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet_search, "Internet Search")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(meta, "Meta")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_app, "Mobile App")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile, "Mobile")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(desktop, "Desktop")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_web, "Mobile Web")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_web_video, "Mobile Web Video")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(online_video, "Online Video")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(paid_social, "Paid Social")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(programmatic, "Programmatic")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(social, "Social")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(yt, "Youtube")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(search, "Search")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(bb_magazines, "B-to-B Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(trade_magazines, "Trade Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(hispanic_newspapers, "Hispanic Newspapers")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_magazines, "Local Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_media, "Local Media")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_tv, "Local TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(span_lang_net_tv, "Span Lang Net TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(syndication, "Syndication")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv_capital, "TV Capital")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv_interior, "TV Interior")

df['Media_type_standarized'] = df['Media_type_standarized'].replace(cable_tv, "Cable TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(open_tv, "Open TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(paid_tv, "Paid TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(free_tv, "Free TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(spot_tv, "Spot TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv, "TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(am_radio, "AM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(fm_radio, "FM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_radio, "Local Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(natl_spot_radio, "Natl Spot Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(network_radio, "Network Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(radio, "Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(cinema, "Cinema")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(print_, "Print")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(newspapers, "Newspapers")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(magazines, "Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(ooh, "OOH")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(digital, "Digital")

df['Media_type_standarized'].unique()

array(['Cable TV', 'FM Radio', 'TV Interior', 'Newspapers', 'AM Radio',
       'TV Capital', 'Magazines', 'Internet', 'OOH', 'Print', 'Cinema',
       'Digital', 'Radio', 'TV', 'Meta', 'Youtube', 'Programmatic',
       'Local Media', 'Open TV', 'Paid TV', 'Social', 'Free TV',
       'Internet Display', 'Trade Magazines', 'Mobile', 'Desktop',
       'Local TV', 'Online display', 'Mobile App', 'Mobile Web Video',
       'Mobile Web', 'Natl Spot Radio', 'Local Radio', 'Spot TV',
       'Online Video', 'Paid Social', 'Internet Search', 'AVOD',
       'B-to-B Magazines', 'Network TV', 'Span Lang Net TV',
       'Hispanic Newspapers', 'Local Magazines', 'Syndication',
       'Network Radio'], dtype=object)

### Month (Number)

In [25]:
# Month (Number)
df["Month_number"] = df["Month"].copy()
df["Month_number"].unique()

array([6, 2, 4, 5, 1, 3, 'April', 'May', 'March', 'January', 'February',
       'June', 'APR', 'FEB', 'JAN', 'JUN', 'MAR', 'MAY', 'JUNIO', 'ABRIL',
       'MAYO', 'FEBRERO', 'MARZO', 'ENERO', 'ABR', 'ENE', 'Enero',
       'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'enero',
       'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
       'March 2023', 'April 2023', 'May 2023', 'January 2023',
       'February 2023', 'June 2023', 'Jan', 'Feb', 'Mar', 'Apr', 'Jun',
       'Jul', Timestamp('2023-01-01 00:00:00'),
       Timestamp('2023-02-01 00:00:00'), Timestamp('2023-03-01 00:00:00'),
       Timestamp('2023-04-01 00:00:00'), Timestamp('2023-05-01 00:00:00'),
       Timestamp('2023-06-01 00:00:00'), 'Ene', 'Abr', 'mar', 'APRIL',
       'JUNE', 'JANUARY', 'FEBRUARY', 'MARCH', 'JUn'], dtype=object)

In [26]:

df.loc[df['Month_number'] == "Jan", 'Month_number'] = 1
df.loc[df['Month_number'] == "JAN", 'Month_number'] = 1
df.loc[df['Month_number'] == "JANUARY", 'Month_number'] = 1
df.loc[df['Month_number'] == "January", 'Month_number'] = 1
df.loc[df['Month_number'] == "Ene", 'Month_number'] = 1
df.loc[df['Month_number'] == "enero", 'Month_number'] = 1
df.loc[df['Month_number'] == "Enero", 'Month_number'] = 1
df.loc[df['Month_number'] == "January 2023", 'Month_number'] = 1

df.loc[df['Month_number'] == "FEB", 'Month_number'] = 2
df.loc[df['Month_number'] == "Feb", 'Month_number'] = 2
df.loc[df['Month_number'] == "Febrero", 'Month_number'] = 2
df.loc[df['Month_number'] == "febrero", 'Month_number'] = 2
df.loc[df['Month_number'] == "FEBRUARY", 'Month_number'] = 2
df.loc[df['Month_number'] == "February", 'Month_number'] = 2
df.loc[df['Month_number'] == "February 2023", 'Month_number'] = 2

df.loc[df['Month_number'] == "MAR", 'Month_number'] = 3
df.loc[df['Month_number'] == "Mar", 'Month_number'] = 3
df.loc[df['Month_number'] == "mar", 'Month_number'] = 3
df.loc[df['Month_number'] == "MARCH", 'Month_number'] = 3
df.loc[df['Month_number'] == "March", 'Month_number'] = 3
df.loc[df['Month_number'] == "Marzo", 'Month_number'] = 3
df.loc[df['Month_number'] == "marzo", 'Month_number'] = 3
df.loc[df['Month_number'] == "March 2023", 'Month_number'] = 3

df.loc[df['Month_number'] == "April 2023", 'Month_number'] = 4
df.loc[df['Month_number'] == "ABR", 'Month_number'] = 4
df.loc[df['Month_number'] == "Abr", 'Month_number'] = 4
df.loc[df['Month_number'] == "Abril", 'Month_number'] = 4
df.loc[df['Month_number'] == "abril", 'Month_number'] = 4
df.loc[df['Month_number'] == "ABRIL", 'Month_number'] = 4
df.loc[df['Month_number'] == "Apr", 'Month_number'] = 4
df.loc[df['Month_number'] == "APR", 'Month_number'] = 4
df.loc[df['Month_number'] == "April", 'Month_number'] = 4
df.loc[df['Month_number'] == "APRIL", 'Month_number'] = 4

df.loc[df['Month_number'] == "MAY", 'Month_number'] = 5
df.loc[df['Month_number'] == "May", 'Month_number'] = 5
df.loc[df['Month_number'] == "Mayo", 'Month_number'] = 5
df.loc[df['Month_number'] == "mayo", 'Month_number'] = 5
df.loc[df['Month_number'] == "MAYO", 'Month_number'] = 5
df.loc[df['Month_number'] == "May 2023", 'Month_number'] = 5

df.loc[df['Month_number'] == "Jun", 'Month_number'] = 6
df.loc[df['Month_number'] == "June", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUNIO", 'Month_number'] = 6
df.loc[df['Month_number'] == "Junio", 'Month_number'] = 6
df.loc[df['Month_number'] == "junio", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUN", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUNE", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUn", 'Month_number'] = 6
df.loc[df['Month_number'] == "June 2023", 'Month_number'] = 6

df.loc[df['Month_number'] == "Julio", 'Month_number'] = 7
df.loc[df['Month_number'] == "julio", 'Month_number'] = 7
df.loc[df['Month_number'] == "Jul", 'Month_number'] = 7

df.loc[df['Month_number'] == "FEBRERO", 'Month_number'] = 2
df.loc[df['Month_number'] == "MARZO", 'Month_number'] = 3
df.loc[df['Month_number'] == "ENERO", 'Month_number'] = 1
df.loc[df['Month_number'] == "ENE", 'Month_number'] = 1

df.loc[df['Month_number'] == pd.Timestamp('2023-01-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-01-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-02-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-02-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-03-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-03-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-04-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-04-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-05-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-05-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-06-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-06-01 00:00:00').month
 
df["Month_number"].unique()

array([6, 2, 4, 5, 1, 3, 7], dtype=object)

### Month (Standardized)

In [27]:
df['Month_standarization'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month_number'].astype(str))

### Top Brand Competitors + ABI

In [28]:
## Null - Not requried


#### Creating Dataframes Market Wise

In [29]:
# Its requried to create number of filtered dataframes for each country to use 
# pivot table to find top competetor, top 10 markets

df_argentina = df[df['Country'] == 'Argentina']
df_bolivia = df[df['Country'] == 'Bolivia']
df_brazil = df[df['Country'] == 'Brazil']
df_canada = df[df['Country'] == 'Canada']
df_chile = df[df['Country'] == 'Chile']
df_china = df[df['Country'] == 'China']
df_colombia = df[df['Country'] == 'Colombia']
df_dom_rep = df[df['Country'] == 'Dominican Republic']
df_ecuador = df[df['Country'] == 'Ecuador']
df_el_salvador = df[df['Country'] == 'El Salvador']
df_guatemala = df[df['Country'] == 'Guatemala']
df_honduras = df[df['Country'] == 'Honduras']
df_india = df[df['Country'] == 'India']
df_japan = df[df['Country'] == 'Japan']
df_mexico = df[df['Country'] == 'Mexico']
df_panama = df[df['Country'] == 'Panama']
df_paraguay = df[df['Country'] == 'Paraguay']
df_peru = df[df['Country'] == 'Peru']
df_south_africa = df[df['Country'] == 'South Africa']
df_south_korea = df[df['Country'] == 'South Korea']
df_uruguay = df[df['Country'] == 'Uruguay']
df_us = df[df['Country'] == 'US']
df_uk = df[df['Country'] == 'UK']
df_vietnam = df[df['Country'] == 'Vietnam']

df_be = df[df['Country'] == 'Belgium']
df_fr = df[df['Country'] == 'France']
df_de = df[df['Country'] == 'Germany']
df_it = df[df['Country'] == 'Italy']
df_nl = df[df['Country'] == 'Netherlands']


markets = [df_argentina, df_bolivia, df_brazil, df_canada, df_chile, df_china, df_colombia, df_dom_rep, df_ecuador, 
           df_el_salvador, df_guatemala, df_honduras, df_india, df_japan, df_mexico, df_panama, df_paraguay, df_peru, 
           df_south_africa, df_south_korea, df_uruguay, df_us, df_uk, df_vietnam, df_be, df_fr, df_de, df_it, df_nl]

print("The Number of markets for Competitive Analysis: {}\n".format(len(markets)))

for market in markets:
    print("The number of data points for country: {}\t\t:{}".format(market["Country"].unique(), market.shape[0]))

The Number of markets for Competitive Analysis: 29

The number of data points for country: ['Argentina']		:175765
The number of data points for country: ['Bolivia']		:6578
The number of data points for country: ['Brazil']		:445
The number of data points for country: ['Canada']		:1772
The number of data points for country: ['Chile']		:296
The number of data points for country: ['China']		:273
The number of data points for country: ['Colombia']		:780
The number of data points for country: ['Dominican Republic']		:66398
The number of data points for country: ['Ecuador']		:441
The number of data points for country: ['El Salvador']		:5711
The number of data points for country: ['Guatemala']		:79229
The number of data points for country: ['Honduras']		:5277
The number of data points for country: ['India']		:162
The number of data points for country: ['Japan']		:234
The number of data points for country: ['Mexico']		:23358
The number of data points for country: ['Panama']		:185
The number of 

### Parent Competitors (Pivots)

In [30]:
## Excluding Other Alcohol category for Competitive analysis of Beer

## Operation steps : 
# 1. filerter datframe by excluding Other Alcohol category
# 2. Use groupby fuction by adv. parent with aggregation with percentage applying lambda fucntion
# 3. Sort values in descending
# 4. Spend_US_Currency column in pivot table would be in percentage

## Function to find top 5 parent competitor in a market in beer category given their ad spend market share in above 5%

def parent_competitors_std(df):
    competitors_df = df[(df['Category_Standardized'] != 'Other Alcohol')]
    competitors_df_gb = competitors_df.groupby([ 'Advertiser_parent_standarized']).agg({'Spend_US_Currency': lambda x: 
                                                                                      round(x.sum()/competitors_df["Spend_US_Currency"].sum()*100,
                                                                                            2)}).sort_values('Spend_US_Currency', 
                                                                                                             ascending = False)
    competitors_df_gb = competitors_df_gb.reset_index()
    competitors_df_gb = (competitors_df_gb[(competitors_df_gb["Spend_US_Currency"] >= 
                                            5) | (competitors_df_gb["Advertiser_parent_standarized"].str.startswith('(ABI)|(HEINEKEN)|(AMSTEL)|(CCU (Heineken Parent))'))])
    
    exceptions = competitors_df_gb["Advertiser_parent_standarized"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Parent_competitor'] = df['Advertiser_parent_standarized'].copy()
    df['Parent_competitor'] = np.where(df['Parent_competitor'].isin(exceptions), 
                                                   df['Parent_competitor'], "All Others")
    print("\n")
    return df['Parent_competitor'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), parent_competitors_std(market)))



['Argentina']:['All Others' 'ABI' 'CCU SA (Heineken Parent)' 'CEPAS']


['Bolivia']:['All Others' 'ABI']


['Brazil']:['ABI' 'HEINEKEN' 'All Others' 'PETROPOLIS']


['Canada']:['MOLSON COORS' 'HEINEKEN' 'All Others' 'MARK ANTHONY BREWING' 'ABI']


['Chile']:['ABI' 'All Others' 'CCU (Heineken Parent)']


['China']:['ABI' 'All Others' 'CARLSBERG' 'CR Beer (Heineken Parent)' 'TSINGTAO'
 'YANJING']


['Colombia']:['All Others' 'ABI' 'CCC (Heineken Parent)']


['Dominican Republic']:['All Others' 'ABI' 'CERVECERIA CENTRO AMERICANA CABLE PANREGIONAL'
 'HEINEKEN' 'COMPAÑIA CERVECERA DE NICARAGUA CABLE PANREGIONAL']


['Ecuador']:['ABI' 'HEINEKEN' 'All Others']


['El Salvador']:['CERVECERIA CENTROAMERICANA' 'ABI' 'CADEJO' 'All Others'
 'CERVECERIA SALVADOREÑA']


['Guatemala']:['HEINEKEN' 'All Others' 'ABI']


['Honduras']:['DISTRIBUIDORA SOLIS' 'ABI' 'CERVECERIA CENTROAMERICANA' 'All Others']


['India']:['UNITED BREWERIES' 'CARLSBERG' 'ABI' 'All Others']


['Japan']:['KIRIN' 'SAPPORO' 'AS

### Top 10 Brands by Market

In [31]:
def top_10_brands(df):
    beer_df = df[(df['Category_Standardized'] != 'Other Alcohol')]
    top10_df = beer_df.groupby([ 'Brand_standarization']).agg({'Spend_US_Currency': 
                                                                            "sum"}).sort_values('Spend_US_Currency',
                                                                                                ascending = False).head(10)
    top10_df = top10_df.reset_index()
        
    exceptions = top10_df["Brand_standarization"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Top10_Brands_by_Market'] = df['Brand_standarization'].copy()
    df['Top10_Brands_by_Market'] = np.where(df['Top10_Brands_by_Market'].isin(exceptions), 
                                                   df['Top10_Brands_by_Market'], "All Others")
    
  
    print("\n")
    return df['Top10_Brands_by_Market'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), top_10_brands(market)))



['Argentina']:['All Others' 'HEINEKEN' 'IMPERIAL' 'BRAHMA' 'ANDES' 'SANTA FE' 'QUILMES'
 'AMSTEL' 'STELLA ARTOIS' 'NORTE' 'DR LEMON']


['Bolivia']:['BURGUESA' 'HUARI' 'PACENA' 'GOLDEN' 'POTOSINA' 'REAL' 'MALTIN' 'UYUNI'
 'All Others']


['Brazil']:['All Others' 'BRAHMA' 'BUDWEISER' 'BUDWEISER ZERO' 'CORONA' 'SKOL'
 'SPATEN' 'AMSTEL' 'HEINEKEN' 'HEINEKEN 0.0' 'ITAIPAVA']


['Canada']:['All Others' 'COORS LIGHT' 'MOLSON PARTNERSHIP' 'MOLSON ULTRA LOW CARB'
 'HEINEKEN' 'WHITE CLAW' 'BLACK FLY COOLERS' 'BUD LIGHT' 'BUDWEISER'
 'MICHELOB ULTRA' 'STELLA ARTOIS']


['Chile']:['All Others' 'AUSTRAL' 'COORS' 'CRISTAL' 'ESCUDO' 'HEINEKEN'
 'ROYAL GUARD' 'SOL' 'STONES' 'BECKER' 'CUSQUENA']


['China']:['BUDWEISER' 'All Others' 'HARBIN' 'BLUE GIRL' 'TUBORG' 'WUSU' 'HEINEKEN'
 'SNOW' 'TSINGTAO' 'YANJING']


['Colombia']:['All Others' 'AGUILA' 'BUDWEISER' 'CLUB COLOMBIA' 'CORONA' 'COSTENA'
 'HEINEKEN' 'POKER' 'ANDINA' 'MILLER' 'AMSTEL']


['Dominican Republic']:['REPUBLICA' 'All Others' 'CLUB COL

### Merging all the files back to main datafram

In [32]:
df_ = pd.DataFrame()
for market in markets:
    df_ = df_.append(market)

### Global Focus Brand

In [33]:
df_['Focused_Global_Brand'] = df_['Brand_standarization'].copy()
global_brands = ['AMSTEL LAGER', 'AMSTEL LIGHT', 'AMSTEL MALTA', 'AMSTEL ULTRA', 'AMSTEL ULTRA SELTZER', 'HEINEKEN',
                 'HEINEKEN 0.0', 'HEINEKEN SILVER']

 
df_['Focused_Global_Brand'] = np.where(df_['Focused_Global_Brand'].isin(global_brands), 
                                                   df_['Focused_Global_Brand'], "All Others")
df_['Focused_Global_Brand'].unique()

array(['All Others', 'HEINEKEN', 'AMSTEL ULTRA', 'HEINEKEN 0.0',
       'HEINEKEN SILVER', 'AMSTEL ULTRA SELTZER', 'AMSTEL LAGER',
       'AMSTEL MALTA', 'AMSTEL LIGHT'], dtype=object)

### Global Parent

In [34]:
df_["Global_Parent"] = df_['Parent_competitor'].copy()

In [35]:
 ## NEED TO DISCUSS WITH BOB

df_.loc[df_["Global_Parent"] == "CCU SA (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CCU (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CR Beer (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CCC (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "VBL (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_["Global_Parent"].unique()

array(['All Others', 'ABI', 'HEINEKEN', 'CEPAS', 'PETROPOLIS',
       'MOLSON COORS', 'MARK ANTHONY BREWING', 'CARLSBERG', 'TSINGTAO',
       'YANJING', 'CERVECERIA CENTRO AMERICANA CABLE PANREGIONAL',
       'COMPAÑIA CERVECERA DE NICARAGUA CABLE PANREGIONAL',
       'CERVECERIA CENTROAMERICANA', 'CADEJO', 'CERVECERIA SALVADOREÑA',
       'DISTRIBUIDORA SOLIS', 'UNITED BREWERIES', 'KIRIN', 'SAPPORO',
       'ASAHI', 'SUNTORY', 'FEDURO', 'BEBIDAS DEL PARAGUAY',
       'EMBOTELLADORA CENTRAL', 'MÓNACO COMERCIAL', 'HITE JINRO',
       'REGIONAL SUR', 'BOSTON BEER CO', 'CONSTELLATION', 'DIAGEO',
       'BREWDOG', 'BEAVERTOWN BREWERY', 'BROWN FORMAN', 'HABECO',
       'SABECO', 'DUVEL', 'SOCIETE COMMERCIALE DE BRASSERIE', 'BAVARIA',
       'FRIESISCHES', 'KROMBACHER', 'PAULANER', 'RADEBERGER', 'VELTINS',
       'WARSTEINER', 'CERES SPA MILANO', 'PERONI SPA BIRRA PERONI ROMA',
       'TEXELSE', 'HUIZEN', 'KONINKLIJKE GROLSCH ENSCHEDE'], dtype=object)

### ABI Parent SOS

In [36]:
df_['ABI_Parent_SOS'] = df_["Global_Parent"].copy()
abi = ['ABI']
df_['ABI_Parent_SOS'] = np.where(df_['ABI_Parent_SOS'].isin(abi), df_['ABI_Parent_SOS'], "All Others")
df_['ABI_Parent_SOS'].unique()

array(['All Others', 'ABI'], dtype=object)

### Heineken Brand SOS

In [37]:
df_['Heineken_Brand_SOS'] = df_['Focused_Global_Brand'].copy()
df_['Heineken_Brand_SOS'].value_counts()

All Others              468284
AMSTEL ULTRA             12454
HEINEKEN                  6283
HEINEKEN SILVER           5348
HEINEKEN 0.0              2077
AMSTEL ULTRA SELTZER       496
AMSTEL LAGER                24
AMSTEL LIGHT                14
AMSTEL MALTA                 6
Name: Heineken_Brand_SOS, dtype: int64

In [38]:
df_['Heineken_Brand_SOS'] = df_['Heineken_Brand_SOS'].str.contains('HEINEKEN')
df_.loc[df_['Heineken_Brand_SOS'] == True, 'Heineken_Brand_SOS'] = 'HEINEKEN'
df_.loc[df_['Heineken_Brand_SOS'] == False, 'Heineken_Brand_SOS'] = 'All Others'
df_['Heineken_Brand_SOS'].value_counts()

All Others    481278
HEINEKEN       13708
Name: Heineken_Brand_SOS, dtype: int64

### Amstel Brand SOS

In [39]:
df_["Amstel_Brand_SOS"] = df_['Focused_Global_Brand'].copy()
df_["Amstel_Brand_SOS"].value_counts()

All Others              468284
AMSTEL ULTRA             12454
HEINEKEN                  6283
HEINEKEN SILVER           5348
HEINEKEN 0.0              2077
AMSTEL ULTRA SELTZER       496
AMSTEL LAGER                24
AMSTEL LIGHT                14
AMSTEL MALTA                 6
Name: Amstel_Brand_SOS, dtype: int64

In [40]:
df_["Amstel_Brand_SOS"] = df_["Amstel_Brand_SOS"].str.contains("AMSTEL")
df_.loc[df_["Amstel_Brand_SOS"] == True, "Amstel_Brand_SOS"] = "AMSTEL"
df_.loc[df_["Amstel_Brand_SOS"] == False, "Amstel_Brand_SOS"] = 'All Others'
df_["Amstel_Brand_SOS"].value_counts()

All Others    481992
AMSTEL         12994
Name: Amstel_Brand_SOS, dtype: int64

### Digital / Traditional

In [41]:
df_["Digital_Traditional"] = df_["Media_Summary"].copy()
df_["Digital_Traditional"].value_counts()

TV         259669
Radio      145096
Digital     75017
OOH         13518
Print        1686
Name: Digital_Traditional, dtype: int64

In [42]:
## Except digital media summrary, rest catagorize as Traditional
digital = ["Digital"]
df_["Digital_Traditional"] = np.where(df_["Digital_Traditional"].isin(digital), df_["Digital_Traditional"], "Traditional") 
df_["Digital_Traditional"].value_counts()

Traditional    419969
Digital         75017
Name: Digital_Traditional, dtype: int64

### Heineken Parent SOS

In [43]:
df_["Heineken_Parent_SOS"] = df_["Global_Parent"].copy()
#df_["Heineken_Parent_SOS"].value_counts()
exception = ["HEINEKEN"]
df_["Heineken_Parent_SOS"] = np.where(df_["Heineken_Parent_SOS"].isin(exception), df_["Heineken_Parent_SOS"], "All Others")
df_["Heineken_Parent_SOS"].value_counts()

All Others    374496
HEINEKEN      120490
Name: Heineken_Parent_SOS, dtype: int64

### Market Group

In [44]:
# Groups as per markets
group_I = ['Argentina', 'Brazil', 'Canada', 'China', 'Colombia', 'India', 'Mexico', 'South Africa', 'South Korea', 'US',
          "Belgium", "France", "Germany", "Ireland", "Italy", "Netherlands", "UK"]
group_II = ["Bolivia", "Chile", 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', 'Japan', 'Panama',
            'Paraguay', 'Peru', 'Uruguay', "Ukraine", 'Vietnam', ]

df_["Market_Group"] = df_["Country"].copy()

# Catgorize markets in two groups
categorize = lambda x: "Group I" if x in group_I else "Group II"
df_["Market_Group"] = df_["Market_Group"].apply(categorize)
df_["Market_Group"].value_counts()

Group I     329077
Group II    165909
Name: Market_Group, dtype: int64

### Brand Rank by Country

In [45]:
brand_beer = df_.loc[df_["Category_Standardized"] != "Other Alcohol"]
beer_pivot = brand_beer.groupby(["Country", "Brand_standarization"]).agg({'Spend_US_Currency': "sum"})
beer_pivot = beer_pivot.reset_index()

beer_pivot_all = pd.DataFrame()
for country in beer_pivot.Country.unique():
    beer_pivot_each = beer_pivot.loc[beer_pivot["Country"] == country]
    beer_pivot_each_ = beer_pivot_each.sort_values("Spend_US_Currency", ascending = False)
    beer_pivot_each_ = beer_pivot_each_.reset_index()
    beer_pivot_each_.drop('index', axis = 1, inplace = True)

    beer_pivot_each_ = beer_pivot_each_.assign(Brand_Rank_by_Country = 
                                           beer_pivot_each_["Spend_US_Currency"].rank(ascending = False))
    
    beer_pivot_all = beer_pivot_all.append(beer_pivot_each_)

beer_pivot_all["Brand_Rank_by_Country"] = beer_pivot_all["Brand_Rank_by_Country"].astype(int)
beer_pivot_all["Brand_Rank_by_Country"] = beer_pivot_all["Brand_Rank_by_Country"].apply(lambda x: 11 if x > 10 else x)
#beer_pivot_all

# Craete another column by combining country & Brand standarised
beer_pivot_all["Country_and_Brand"] = beer_pivot_all["Country"] + "_" + beer_pivot_all["Brand_standarization"]

# Only considering top 10 brand ranks rest can be catagorize as 11 later
beer_pivot_all = beer_pivot_all.loc[beer_pivot_all["Brand_Rank_by_Country"] < 11]
beer_pivot_all = beer_pivot_all[["Country_and_Brand", "Brand_Rank_by_Country"]]

#Create unique column by merge country & Brand standarised
df_["Country_and_Brand"] = df_["Country"] + "_" + df_["Brand_standarization"]

## Work on the master datafram, vlookup - by merge function left join:
df_rank = pd.merge(df_, beer_pivot_all, on = "Country_and_Brand", how = "left") 

## if NaN values in the Brand_Rank_by_Country_y column, replace the values as 11th rank i.e. 11
df_rank["Brand_Rank_by_Country_y"] = df_rank["Brand_Rank_by_Country_y"].fillna(11)
df_rank["Brand_Rank_by_Country_y"] = df_rank["Brand_Rank_by_Country_y"].astype(int) # Convert rank into int from float

df_rank.drop(['Brand_Rank_by_Country_x', 'Country_and_Brand'], axis = 1, inplace = True)
df_rank = df_rank.rename(columns = {'Brand_Rank_by_Country_y': 'Brand_Rank_by_Country'})


In [46]:
df_final = df_rank[['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency', 'Category_Standardized',
       'Advertiser_parent_standarized', 'Brand_standarization',
       'Parent_competitor', 'Top_Brand_Competitors_and_ABI',
       'Focused_Brand_Competitors', 'Top10_Brands_by_Market', 'Media_Summary',
       'Media_type_standarized', 'Month_number', 'Month_standarization',
       'Focused_Global_Brand', 'Global_Parent', 'ABI_Parent_SOS',
       'Heineken_Brand_SOS', 'Amstel_Brand_SOS', 'Digital_Traditional',
       'Heineken_Parent_SOS', 'Brand_Rank_by_Country', 'Market_Group']]

In [47]:
### Dividing data into two fiels based on Groups:
df_final_1 = df_final[df_final["Market_Group"] == "Group I"]
df_final_2 = df_final[df_final["Market_Group"] == "Group II"]

In [48]:
# Initial Dataframe, Final dataframe shape
print(df_initial.shape)
print(df_final.shape)
print(df_final_1.shape)
print(df_final_2.shape)

(494986, 11)
(494986, 31)
(329077, 31)
(165909, 31)


**** DO NOT EXECUTE CODE AFTER THIS LINE***

### CODE TESTING

In [49]:
os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Processed Data")
print(os.getcwd())
#df_final.to_excel("Q2-Competitive Ad Spend-V4.xlsx", index=False)

#df_final.to_excel("Q2-Competitive Ad Spend-Adv test.xlsx", index=False) # test

df_final_1.to_excel("Q2-Competitive Ad Spend-Group I.xlsx", index=False)
df_final_2.to_excel("Q2-Competitive Ad Spend-Group II.xlsx", index=False)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")
print(f"Elapsed time: {elapsed_time/60} minutes")

C:\ABI\Q2_Competitive\Ad_Spend\Processed Data
Elapsed time: 429.23363971710205 seconds
Elapsed time: 7.153893995285034 minutes


In [50]:
## 700+ lines of code
## the whole Code excecution take 10 - 25 min apx